In [210]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pickle
import pandas as pd
import time

def scrape(sub_red):
    token = 'dLN3qq4_CCZeNJTAPTwGdA'
    reddit_url = 'https://www.reddit.com/r/' + sub_red + '/new/'
    api = CrawlingAPI({'token': token})
    response = api.get(reddit_url)

    if response['status_code'] == 200:
        return response['body']
    else:
        print('Error')
html_data = scrape('depression')

In [230]:
def save_post(html_data,sub_red):    
    with open('model.p', 'rb') as file:
        model = pickle.load(file)
    with open('vectorizer.p', 'rb') as file:
        vectorizer = pickle.load(file)
    soup = BeautifulSoup(html_data, 'html.parser')
    instances = soup.find_all('div', property="schema:articleBody")
    docs = []
    for instance in instances:
        for p_tag in instance:
            if p_tag:
                docs.append(''.join(p_tag.get_text().strip()))

    verdicts = pd.read_csv('verdicts.csv')
    for doc in docs:
        vectorized = vectorizer.transform([doc])
        predict_probas = model.predict_proba(vectorized)[:, 1]
        threshold = 0.95
        bool = (predict_probas >= threshold)
        if not (verdicts['post'] == doc).any() and doc:
        # if doc:
            verdicts = pd.concat([verdicts, pd.DataFrame({'post':doc,'verdict':bool, 'sub_red':sub_red})])
            verdicts.to_csv('verdicts.csv', index=False)
            print('saved a post from r/' + sub_red + ' ' + str(bool))
save_post(html_data,'confessions')



saved a post from r/confessions [False]
saved a post from r/confessions [False]
saved a post from r/confessions [ True]
saved a post from r/confessions [False]
saved a post from r/confessions [ True]


In [209]:
sub_reds = ['depression', 'sad', 'confessions']
# sub_reds = ['confessions']
iterations = 0
# while iterations < 24:

for sub_red in sub_reds:
    try: 
        save_post(scrape(sub_red),sub_red)
    except Exception as e: print(sub_red, e)
    # time.sleep(10)
# time.sleep(1800)

iterations += 1

saved a post from r/depression [False]
saved a post from r/depression [ True]
saved a post from r/depression [ True]
saved a post from r/depression [False]
saved a post from r/depression [ True]
saved a post from r/sad [False]
saved a post from r/sad [ True]
saved a post from r/sad [False]
saved a post from r/sad [ True]
saved a post from r/sad [False]
saved a post from r/sad [False]
saved a post from r/confessions [ True]


In [229]:
verdicts = pd.DataFrame({'sub_red': [],'verdict':[], 'post':[]})
verdicts.to_csv('verdicts.csv')

KeyboardInterrupt: 